In [41]:
# %% ========================================
# 0. ライブラリ & データ読み込み
# ========================================
import pandas as pd
import numpy as np
import os

# 01_exploration の出力を読み込み
VALID_PATH = "/workspace/data/valid/StatsAllGroup_cleaned_valid.parquet"
df_valid = pd.read_parquet(VALID_PATH)
print(f"✅ Loaded valid data: {df_valid.shape[0]} rows, {df_valid['athlete_name'].nunique()} athletes")

✅ Loaded valid data: 66950 rows, 57 athletes


In [42]:
print("\n🔍 データ型ごとの列名一覧:")

dtype_map = {}

for col, dtype in df_valid.dtypes.items():
    dtype_map.setdefault(str(dtype), []).append(col)

for dtype, cols in dtype_map.items():
    print(f"\n--- {dtype} ({len(cols)} columns) ---")
    for c in cols:
        print(" -", c)




🔍 データ型ごとの列名一覧:

--- datetime64[ns] (3 columns) ---
 - date_
 - start_dt
 - end_dt

--- float64 (50 columns) ---
 - session_duration_sec
 - start_time
 - end_time
 - total_duration
 - total_distance
 - max_heart_rate
 - mean_heart_rate
 - min_heart_rate
 - max_vel
 - velocity_band1_total_distance
 - velocity_band2_total_distance
 - velocity_band3_total_distance
 - velocity_band4_total_distance
 - velocity_band5_total_distance
 - velocity_band6_total_distance
 - gen2_velocity_band1_total_effort_count
 - gen2_velocity_band2_total_effort_count
 - velocity2_band6_total_distance
 - velocity2_band7_total_distance
 - velocity2_band8_total_distance
 - heart_rate_band2_total_duration
 - heart_rate_band3_total_duration
 - total_player_load
 - total_jumps
 - total_dive_load_centre
 - total_dive_load_left
 - total_dive_load_right
 - dive_right_count
 - dive_left_count
 - dive_centre_count
 - total_dives_centre
 - total_dives_left
 - total_dives_right
 - total_time_to_feet_centre
 - total_time_to_f

In [43]:
# ============================================================
# 日次集計：df_valid（セッション単位） → daily_raw（1日1行）
# ============================================================

df = df_valid.copy()

# ------------- 1. 集計対象の自動判別 ------------- #

numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()

# 除外（集計に使わない）
exclude_cols = ["start_time", "end_time"]  # ← total_duration は残す方が有益

numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

# 集計方法の辞書
agg_dict = {}

for col in numeric_cols:
    if col == "mean_heart_rate":
        agg_dict[col] = "mean"
    elif col in ["max_heart_rate", "max_vel"]:
        agg_dict[col] = "max"
    elif col == "min_heart_rate":
        agg_dict[col] = "min"
    else:
        # デフォルトは日次合計
        agg_dict[col] = "sum"


# ------------- 2. 日次集計の実行 ------------- #

daily_raw = (
    df.groupby(["athlete_name", "date_"])
      .agg(agg_dict)
      .reset_index()
      .sort_values(["athlete_name", "date_"])
)

print("✅ 日次集計完了！")
print(daily_raw.head())


✅ 日次集計完了！
  athlete_name      date_  session_duration_sec  total_duration  \
0   Asahi YADA 2024-01-12                4034.0          4034.0   
1   Asahi YADA 2024-01-13                3590.0          3590.0   
2   Asahi YADA 2024-01-18                6642.0          6642.0   
3   Asahi YADA 2024-01-19                5308.0          5308.0   
4   Asahi YADA 2024-01-20                1823.0          1823.0   

   total_distance  max_heart_rate  mean_heart_rate  min_heart_rate   max_vel  \
0      4025.45282           169.0        98.472747             0.0  21.05945   
1      3439.64703           175.0       127.201588             0.0  17.87283   
2      7629.91396           152.0        94.724526             0.0  24.59469   
3      6074.19409           120.0        81.834778             0.0  25.29740   
4      4119.96143            84.0        77.995880             0.0  24.51995   

   velocity_band1_total_distance  ...  ima_band2_jump_count  \
0                     1508.65997  ...      

In [44]:
# ============================================================
# ACWR（EWMA版）計算関数
# ============================================================

def compute_acwr_ewma(series, acute_span=7, chronic_span=28):
    acute = series.ewm(span=acute_span, adjust=False).mean()
    chronic = series.ewm(span=chronic_span, adjust=False).mean()
    acwr = acute / chronic
    acwr.iloc[:chronic_span] = np.nan
    return acwr

def add_acwr_features(df, var, acute=7, chronic=28):
    g = df.groupby("athlete_name")[var]
    df[f"EWMA{acute}_{var}"] = g.transform(lambda s: s.ewm(span=acute, adjust=False).mean())
    df[f"EWMA{chronic}_{var}"] = g.transform(lambda s: s.ewm(span=chronic, adjust=False).mean())
    df[f"ACWR_EWMA_{var}"] = g.transform(lambda s: compute_acwr_ewma(s, acute, chronic))
    df[f"EWMA_ratio_{var}"] = df[f"EWMA{acute}_{var}"] / df[f"EWMA{chronic}_{var}"]
    return df

# ============================================================
# ACWRを追加する対象
# ============================================================

target_vars = ["total_distance", "total_player_load"]

for var in target_vars:
    daily_raw = add_acwr_features(daily_raw, var)

print("✅ ACWR / EWMA 特徴量を daily_raw に追加完了！")
print(daily_raw.filter(regex="ACWR|EWMA").head())

# ============================================================
# 保存
# ============================================================

DAILY_DIR = "/workspace/data/daily"
os.makedirs(DAILY_DIR, exist_ok=True)

# ファイル名を2つに分けて明確化
daily_csv_path = f"{DAILY_DIR}/StatsAllGroup_cleaned_valid_daily.csv"
daily_parquet_path = f"{DAILY_DIR}/StatsAllGroup_cleaned_valid_daily.parquet"

# 保存
daily_raw.to_csv(daily_csv_path, index=False)
daily_raw.to_parquet(daily_parquet_path, index=False)

print("💾 Saved:")
print(" - CSV:", daily_csv_path)
print(" - Parquet:", daily_parquet_path)


✅ ACWR / EWMA 特徴量を daily_raw に追加完了！
   EWMA7_total_distance  EWMA28_total_distance  ACWR_EWMA_total_distance  \
0           4025.452820            4025.452820                       NaN   
1           3879.001372            3985.052421                       NaN   
2           4816.729519            4236.422182                       NaN   
3           5131.095662            4363.165072                       NaN   
4           4878.312104            4346.392407                       NaN   

   EWMA_ratio_total_distance  EWMA7_total_player_load  \
0                   1.000000               458.783570   
1                   0.973388               445.660230   
2                   1.136981               571.215605   
3                   1.176003               610.135531   
4                   1.122382               576.485063   

   EWMA28_total_player_load  ACWR_EWMA_total_player_load  \
0                458.783570                          NaN   
1                455.163338                 

In [45]:
print("\n🔍 ACWR / EWMA 系列の列（存在チェック）")
acwr_cols = [c for c in daily_raw.columns if "ACWR" in c or "EWMA" in c]
print(acwr_cols)
print(f"📌 合計 {len(acwr_cols)} 列")



🔍 ACWR / EWMA 系列の列（存在チェック）
['EWMA7_total_distance', 'EWMA28_total_distance', 'ACWR_EWMA_total_distance', 'EWMA_ratio_total_distance', 'EWMA7_total_player_load', 'EWMA28_total_player_load', 'ACWR_EWMA_total_player_load', 'EWMA_ratio_total_player_load']
📌 合計 8 列


In [46]:
print("\n🔍 ACWR / EWMA 先頭5行")
display(daily_raw[acwr_cols].head())



🔍 ACWR / EWMA 先頭5行


,EWMA7_total_distance,EWMA28_total_distance,ACWR_EWMA_total_distance,EWMA_ratio_total_distance,EWMA7_total_player_load,EWMA28_total_player_load,ACWR_EWMA_total_player_load,EWMA_ratio_total_player_load
0,4025.452820,4025.452820,NaN,1.000000,458.783570,458.783570,NaN,1.000000
1,3879.001372,3985.052421,NaN,0.973388,445.660230,455.163338,NaN,0.979122
2,4816.729519,4236.422182,NaN,1.136981,571.215605,489.143917,NaN,1.167786
3,5131.095662,4363.165072,NaN,1.176003,610.135531,505.540565,NaN,1.206897
4,4878.312104,4346.392407,NaN,1.122382,576.485063,503.471123,NaN,1.145021


In [47]:
print("\n📌 全列数:", len(daily_raw.columns))
daily_raw.info()



📌 全列数: 68
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16499 entries, 0 to 16498
Data columns (total 68 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   athlete_name                                    16499 non-null  object        
 1   date_                                           16499 non-null  datetime64[ns]
 2   session_duration_sec                            16499 non-null  float64       
 3   total_duration                                  16499 non-null  float64       
 4   total_distance                                  16499 non-null  float64       
 5   max_heart_rate                                  16499 non-null  float64       
 6   mean_heart_rate                                 16499 non-null  float64       
 7   min_heart_rate                                  16499 non-null  float64       
 8   max_vel                            

In [48]:
# ============================================================
# 動的・静的・ACWR の3分割
# ============================================================

dynamic_cols = [
    "total_distance",
    "total_player_load",
    "max_vel",
    "mean_heart_rate",
    "max_heart_rate",
    "heart_rate_band2_total_duration",
    "heart_rate_band3_total_duration",
    "velocity_band4_total_distance",
    "velocity_band5_total_distance",
    "velocity_band6_total_distance",
]

acwr_cols = [
    "EWMA7_total_distance",
    "EWMA28_total_distance",
    "ACWR_EWMA_total_distance",
    "EWMA_ratio_total_distance",
    "EWMA7_total_player_load",
    "EWMA28_total_player_load",
    "ACWR_EWMA_total_player_load",
    "EWMA_ratio_total_player_load",
]

# ---------- 全列 ----------
all_cols = daily_raw.columns.tolist()

# ---------- 静的 = 全部から dynamic + acwr + id列 を除いたもの ----------
static_cols = [
    c for c in all_cols
    if c not in dynamic_cols
    and c not in acwr_cols
    and c not in ["athlete_name", "date_"]
]

# データフレーム作成
df_dynamic = daily_raw[["athlete_name", "date_"] + dynamic_cols].copy()
df_acwr    = daily_raw[["athlete_name", "date_"] + acwr_cols].copy()
df_static  = daily_raw[["athlete_name", "date_"] + static_cols].copy()

print("📌 分割結果:")
print(" dynamic:", df_dynamic.shape)
print(" static :", df_static.shape)
print(" acwr   :", df_acwr.shape)

# ------------------------------------------------------------
# 保存
# ------------------------------------------------------------
SPLIT_DIR = "/workspace/data/daily/split"
os.makedirs(SPLIT_DIR, exist_ok=True)

df_dynamic.to_parquet(f"{SPLIT_DIR}/dynamic_features.parquet", index=False)
df_static.to_parquet(f"{SPLIT_DIR}/static_features.parquet", index=False)
df_acwr.to_parquet(f"{SPLIT_DIR}/acwr_features.parquet", index=False)

print("\n💾 Saved split datasets successfully!")


📌 分割結果:
 dynamic: (16499, 12)
 static : (16499, 50)
 acwr   : (16499, 10)

💾 Saved split datasets successfully!
